---
# Import Libraries

In [1]:
import os
import sys

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
sys.path.append("../../../deep-learning-dna")
sys.path.append("../")
sys.path.append("../../../deep-learning-dna/common")

In [4]:
import wandb

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import math
import string

from Attention import Set_Transformer
from common.models import dnabert
from common import dna
from lmdbm import Lmdb
from common.data import DnaSequenceGenerator, DnaLabelType, DnaSampleGenerator, find_dbs
import wandb

import tf_utils as tfu

In [6]:
strategy = tfu.devices.select_gpu(0, use_dynamic_memory=True)

---
# Load Data

In [7]:
def gen_data(batch_size, length=5):
    x = np.random.randint(1, 100, (batch_size, length))
    y = np.max(x, axis=1)
    return x, y # (batch_size, length)

In [8]:
x, y = gen_data(3, 5)

In [9]:
print(x.shape, y.shape)

(3, 5) (3,)


In [10]:
x

array([[91, 88, 22, 47, 97],
       [32, 58, 72, 21, 15],
       [44, 58,  1, 20, 86]])

In [11]:
y

array([97, 72, 86])

---
# Cache Memory

In [12]:
def Cache_Memory(current_state, previous_state, memory_length):
    
    if memory_length is None or memory_length == 0:
        return None
    else:
        if previous_state is None:
            new_mem = current_state[:, -memory_length:, :]
        else:
            new_mem = tf.concat(
                    [previous_state, current_state], 1)[:, -memory_length:, :]

    return tf.stop_gradient(new_mem)

---
# Attention

In [13]:
class Attention(keras.Model):
    def __init__(self, num_induce, embed_dim, num_heads, use_layernorm, pre_layernorm, use_keras_mha):
        super(Attention, self).__init__()
        
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        if self.num_induce == 0:       
            self.attention = (Set_Transformer.SetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm,use_keras_mha=self.use_keras_mha))
        else:
            self.attention = Set_Transformer.InducedSetAttentionBlock(embed_dim=self.embed_dim, num_heads=self.num_heads, num_induce=self.num_induce, use_layernorm=self.use_layernorm, pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha)
    
    
    def call(self, data, mems):
                
            attention = self.attention([data, mems])
                
            return attention

---
# XL Block

In [14]:
class TransformerXLBlock(tf.keras.layers.Layer):
    def __init__(self,
                 num_induce, 
                 embed_dim,
                 num_heads,
                 use_layernorm,
                 pre_layernorm,
                 use_keras_mha,):

        super(TransformerXLBlock, self).__init__()
        
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.attention = Attention
        
        self.attention_layer = self.attention(self.num_induce, self.embed_dim, self.num_heads, self.use_layernorm, self.pre_layernorm, self.use_keras_mha)

   
    def call(self,
             content_stream,
             state=None):
        
        attention_output = self.attention_layer(content_stream, state)

        return attention_output

---
# Transformer XL

In [15]:
class TransformerXL(tf.keras.layers.Layer):
    def __init__(self,
                 mem_switched, 
                 num_layers,
                 num_induce,
                 embed_dim,
                 num_heads,
                 dropout_rate,
                 mem_len=None,
                 use_layernorm=True,
                 pre_layernorm=True, 
                 use_keras_mha=True):
        
        super(TransformerXL, self).__init__()

        self.mem_switched = mem_switched
        self.num_layers = num_layers
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate
        self.mem_len = mem_len
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha

        self.transformer_xl_layers = []
        
        for i in range(self.num_layers):
            self.transformer_xl_layers.append(
                    TransformerXLBlock(self.num_induce,
                                        self.embed_dim,
                                        self.num_heads,
                                        self.use_layernorm,
                                        self.pre_layernorm, 
                                        self.use_keras_mha))

        self.output_dropout = tf.keras.layers.Dropout(rate=self.dropout_rate)

    def call(self,
             content_stream,
             state=None):
        
        new_mems = []

        if state is None:
            state = [None] * self.num_layers
            
        for i in range(self.num_layers):
            if self.mem_switched == False:
                new_mems.append(Cache_Memory(content_stream, state[i], self.mem_len))
            
            transformer_xl_layer = self.transformer_xl_layers[i]
            
            transformer_xl_output = transformer_xl_layer(content_stream=content_stream,
                                                        state=state[i])
            
            content_stream = self.output_dropout(transformer_xl_output)
            
            if self.mem_switched == True:
                new_mems.append(Cache_Memory(content_stream, state[i], self.mem_len))

        output_stream = content_stream
        return output_stream, new_mems

---
# Xl Model Class

In [16]:
class XlModel(keras.Model):
    def __init__(self, mem_switched, max_files, encoder, block_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, mem_len, dropout_rate, num_seeds, use_layernorm, pre_layernorm, use_keras_mha):
        super(XlModel, self).__init__()
        
        self.mem_switched = mem_switched
        self.max_files = max_files
        self.encoder = encoder
        self.block_size = block_size
        self.max_set_len = max_set_len
        self.num_induce = num_induce
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_heads = num_heads
        self.mem_len = mem_len
        self.dropout_rate = dropout_rate
        self.num_seeds = num_seeds
        self.use_layernorm = use_layernorm
        self.pre_layernorm = pre_layernorm
        self.use_keras_mha = use_keras_mha
        
        self.linear_layer = keras.layers.Dense(self.embed_dim)
        
        self.transformer_xl = TransformerXL(self.mem_switched,
                                            self.num_layers,
                                             self.num_induce,
                                             self.embed_dim,
                                             self.num_heads,
                                             self.dropout_rate,
                                             self.mem_len,
                                             self.use_layernorm,
                                             self.pre_layernorm,
                                             self.use_keras_mha)
        

        self.segment_pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=self.num_seeds,embed_dim=self.embed_dim,num_heads=self.num_heads,use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha, is_final_block=True)
       
        self.output_pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=self.num_seeds,embed_dim=self.embed_dim,num_heads=self.num_heads,use_layernorm=self.use_layernorm,pre_layernorm=self.pre_layernorm, use_keras_mha=self.use_keras_mha, is_final_block=True)
    
        self.dropout_layer = keras.layers.Dropout(.5)
    
        self.dense_layer = keras.layers.Dense(1) 
        
        #Classification
        #self.dense_layer = keras.layers.Dense(100) 

    def call(self, x, training=None):    
 
        mems = tf.zeros((self.num_layers, tf.shape(x)[0], self.mem_len, self.embed_dim))
    
        x = tf.expand_dims(x, axis=2)
        
        linear_transform = self.linear_layer(x)

        for i in range(0, self.max_set_len, self.block_size):

            block = linear_transform[:,i:i+self.block_size]
            
            output, mems = self.transformer_xl(content_stream=block, state=mems)
            
            segment_pooling = self.segment_pooling_layer(output)

        output_pooling = self.output_pooling_layer(segment_pooling)
        
        dropout = self.dropout_layer(output_pooling)

        dense = self.dense_layer(dropout)

        #output = tf.reshape(dense, tf.shape(dense)[:2])
        
        #Classification
        output = tf.reshape(dense, tf.shape(dense)[::2])         
        
        return output

---
# Xl Parameters

In [17]:
#Xl Parameters
max_files = 0
encoder = 0
mem_switched = True
num_induce = 0
embed_dim = 64
num_layers = 4
num_heads = 8
mem_len = 0
dropout_rate = 0.01
num_seeds = 1
use_layernorm = True
pre_layernorm = True
use_keras_mha = True

---
# Create Models

In [18]:
block_size = 20
length = 20

In [19]:
x, y = gen_data(batch_size=10, length=length)
vx, vy = gen_data(batch_size=10, length=length)

In [20]:
max_set_len = length
set_len = length

In [21]:
model = XlModel(mem_switched, max_files, encoder, block_size, max_set_len, num_induce, embed_dim, num_layers, num_heads, mem_len, dropout_rate, num_seeds, use_layernorm, pre_layernorm, use_keras_mha)
model.compile(optimizer=keras.optimizers.Adam(1e-3),loss=keras.losses.MeanAbsoluteError())

In [22]:
y[:5]

array([97, 97, 97, 91, 99])

In [23]:
model.predict(x[:5])

array([[-0.39295352],
       [-0.47098017],
       [-0.37351608],
       [-0.40641773],
       [-0.4280383 ]], dtype=float32)

In [24]:
epochs = 500

In [25]:
history = model.fit(x, y, validation_data=(vx, vy), epochs=epochs, verbose=1)

Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 97.3646 - val_loss: 89.4729
Epoch 2/500
1/1 [==============================] - 0s 61ms/step - loss: 92.2015 - val_loss: 87.8514
Epoch 3/500
1/1 [==============================] - 0s 63ms/step - loss: 89.9914 - val_loss: 87.1742
Epoch 4/500
1/1 [==============================] - 0s 67ms/step - loss: 90.3208 - val_loss: 86.6127
Epoch 5/500
1/1 [==============================] - 0s 67ms/step - loss: 88.8447 - val_loss: 86.1644
Epoch 6/500
1/1 [==============================] - 0s 77ms/step - loss: 89.8347 - val_loss: 85.7099
Epoch 7/500
1/1 [==============================] - 0s 73ms/step - loss: 89.1812 - val_loss: 85.2104
Epoch 8/500
1/1 [==============================] - 0s 67ms/step - loss: 87.8810 - val_loss: 84.7157
Epoch 9/500
1/1 [==============================] - 0s 70ms/step - loss: 87.7617 - val_loss: 84.3576
Epoch 10/500
1/1 [==============================] - 0s 72ms/step - loss: 87.1385 - val_loss: 84.1147
E

In [174]:
#Training

In [200]:
x[:5]

array([[47,  4, 46, 35,  6, 66, 25, 20, 20, 59, 17, 61,  3, 81, 92, 43,
        98, 84, 16, 49],
       [59,  6, 56, 95, 81, 41, 90, 98, 99, 28, 74, 27, 40, 44, 88, 82,
        34, 35, 52, 97],
       [29, 78, 51, 59, 23, 50, 42, 72, 96, 62, 44, 90, 87, 61, 85, 13,
        23, 44, 87, 36],
       [94, 56, 42, 58, 13, 69, 52, 21, 20, 84, 33, 66, 88, 91, 75, 11,
        76, 36, 87, 94],
       [10, 74, 39, 99, 41, 93, 37, 95,  9, 77, 72, 63, 84, 31, 99, 63,
        94, 37,  3, 28]])

In [26]:
y[:5]

array([97, 97, 97, 91, 99])

In [27]:
model.predict(x[:5])

array([[40.858913],
       [40.858963],
       [40.85885 ],
       [40.858837],
       [40.858932]], dtype=float32)

In [177]:
#tf.argmax(model.predict(x[:5]), axis = 1)

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>

In [178]:
#Validation

In [179]:
x, y = gen_data(batch_size=30, length=10)

In [ ]:
y[:5]

array([90, 92, 76, 95, 81])

In [181]:
tf.argmax(model.predict(x[:5]), axis = 1)

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 0, 0])>